# OpenAI重大更新

## 2023-06-13 函数调用和其他API更新

[Function calling and other API updates](https://openai.com/blog/function-calling-and-other-api-updates)

一些令人兴奋的更新：

- 在Chat Completions API中新增了函数调用功能。
- 更新了更可控的gpt-4和gpt-3.5-turbo版本。
- 新增了gpt-3.5-turbo的16k上下文版本（相对于标准的4k版本）。
- 我们的最先进的嵌入模型成本降低了75%。
- gpt-3.5-turbo的输入令牌成本降低了25%。
- 公布了gpt-3.5-turbo-0301和gpt-4-0314模型的停用时间表。

**函数调用**功能使我们能够利用模型的自然语言理解能力，将人类语言有效地转化为结构化数据或在我们的代码中进行特定的函数调用。

### 基于OpenAI的Python SDK使用函数调用

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
model = 'gpt-3.5-turbo-0613'

In [3]:
function_descriptions = [
  {
      "name": "get_student_score",
      "description": "Get the student score by given his or her name",
      "parameters": {
          "type": "object",
          "properties": {
              "name": {
                  "type": "string",
                  "description": "The student's name",
              }
          },
          "required": ["name"],
      },
    }
]

In [4]:
user_query = "What's the performance of Lucy in the school this year?"

In [5]:
import openai

In [6]:
response = openai.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": user_query}],
    functions=function_descriptions,
    function_call="auto",
)

In [7]:
ai_response_message = response.choices[0].message
print(ai_response_message)

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n"name": "Lucy"\n}', name='get_student_score'), tool_calls=None)


In [8]:
name = eval(ai_response_message.function_call.arguments).get("name")

In [9]:
import json

SCORES = { 'Alex': 90, 'Lucy': 60 }
def get_student_score(name):

    """Get the student score by given his or her name"""

    score = {
        "name": name,
        "score": SCORES[name]
    }
    return json.dumps(score)

In [10]:
function_response = get_student_score(name=name)

function_response

'{"name": "Lucy", "score": 60}'

In [11]:
second_response = openai.chat.completions.create(
    model=model,
    messages=[
        {"role": "user", "content": user_query},
        ai_response_message,
        {
            "role": "function",
            "name": "get_student_score",
            "content": function_response,
        },
    ],
)

In [12]:
print (second_response.choices[0].message.content)

Lucy's performance in school this year is a score of 60.


### 基于LangChain框架使用函数调用

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, ChatMessage
from langchain.tools.render import format_tool_to_openai_function
from langchain.tools.youtube.search import YouTubeSearchTool

In [1]:
llm = ChatOpenAI(model=model)

NameError: name 'ChatOpenAI' is not defined

#### LangChain工具(Tool)的OpenAI函数调用能力

In [34]:
tools = [YouTubeSearchTool()]
functions = [format_tool_to_openai_function(t) for t in tools]

In [35]:
functions

[{'name': 'youtube_search',
  'description': 'search for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional',
  'parameters': {'properties': {'__arg1': {'title': '__arg1',
     'type': 'string'}},
   'required': ['__arg1'],
   'type': 'object'}}]

In [36]:
message = llm.predict_messages([HumanMessage(content='search videos in the topic of OpenAI on Youtube')], functions=functions)

In [37]:
message.additional_kwargs

{'function_call': {'arguments': '{\n  "__arg1": "OpenAI"\n}',
  'name': 'youtube_search'}}

#### LangChain使用OpenAI函数调用实例

In [40]:
function_descriptions = [
    {
        "name": "remove_word_from_string",
        "description": "Remove a word from a string by given its index",
        "parameters": {
            "type": "object",
            "properties": {
                "string": {
                    "type": "string",
                    "description": "The original string to be processed",
                },
                "index": {
                    "type": "integer",
                    "description": "The index of the word to be removed"
                },
            },
            "required": [
                "string",
                "index"
            ],
        },
    },
    {
        "name": "send_message_by_email",
        "description": "Send an email with the text message to a recipient",
        "parameters": {
            "type": "object",
            "properties": {
                "recipient": {
                    "type": "string",
                    "description": "The email address of the recipient",
                },
                "message": {
                    "type": "string",
                    "description": "The message of the email content",
                }
            },
            "required": [
                "recipient",
                "message"
            ],
        },
    }
]


In [39]:
question = """
I have a string as follows:

black yellow red blue green

Please do the following 2 operations on it:
1. Remove the third word in the string
2. Send the updated string to Alex via email alex@xyz.com
"""

Helper functions to get function parameter names. They will be used to implement dynamic function calls in Python.

In [44]:
import inspect

def get_function_parameter_names(function):
  if function is not None and inspect.isfunction(function):
      parameter_names = inspect.signature(function).parameters.keys()
      return list(parameter_names)
  else:
      return None

In [45]:
def remove_word_from_string(string, index):
    words = string.split()

    if 0 <= index < len(words):
        del words[index]

        return ' '.join(words)
    else:
        return string

def send_message_by_email(recipient, message):
    print(f'Sending {message} to {recipient}')
    return f'Just sent email to {recipient}'

In [46]:
parameter_names = get_function_parameter_names(remove_word_from_string)
parameter_names

['string', 'index']

In [41]:
first_response = llm.predict_messages([HumanMessage(content=question)], functions=function_descriptions)
first_response

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n"string": "black yellow red blue green",\n"index": 2\n}', 'name': 'remove_word_from_string'}})

In [42]:
first_response.additional_kwargs

{'function_call': {'arguments': '{\n"string": "black yellow red blue green",\n"index": 2\n}',
  'name': 'remove_word_from_string'}}

In [47]:
# Get function name, and its arguments

function_name = first_response.additional_kwargs["function_call"]["name"]
arguments = json.loads(first_response.additional_kwargs["function_call"]["arguments"])

# Locate the function and make the call
the_function = globals().get(function_name)
parameter_names = get_function_parameter_names(the_function)
parameter_values = []
for parameter_name in parameter_names:
  parameter_values.append(arguments[parameter_name])

returned_value = the_function(*parameter_values)

In [48]:
print(returned_value)

black yellow blue green


In [49]:
second_response = llm.predict_messages(
    [
        HumanMessage(content=question),
        AIMessage(content=str(first_response.additional_kwargs)),
        ChatMessage(
            role='function',
            additional_kwargs = {'name': function_name},
            content = returned_value
        )
    ],
    functions=function_descriptions
)

In [50]:
second_response.additional_kwargs

{'function_call': {'arguments': '{\n"recipient": "alex@xyz.com",\n"message": "black yellow blue green"\n}',
  'name': 'send_message_by_email'}}

In [53]:
second_response.content

''

In [54]:
# Again get function name, and its arguments

function_name = second_response.additional_kwargs["function_call"]["name"]
arguments = json.loads(second_response.additional_kwargs["function_call"]["arguments"])

# Locate the function and make the call
the_function = globals().get(function_name)
parameter_names = get_function_parameter_names(the_function)
parameter_values = []
for parameter_name in parameter_names:
  parameter_values.append(arguments[parameter_name])

returned_value = the_function(*parameter_values)

Sending black yellow blue green to alex@xyz.com


In [55]:
# Print the returned value in the second function call
print(returned_value)

Just sent email to alex@xyz.com


In [56]:
third_response = llm.predict_messages(
    [
        HumanMessage(content=question),
        AIMessage(content=str(first_response.additional_kwargs)),
        AIMessage(content=str(second_response.additional_kwargs)),
        ChatMessage(
            role='function',
            additional_kwargs = {'name': function_name},
            content = returned_value
        )
    ], functions=function_descriptions
)

In [57]:
third_response

AIMessage(content='I have removed the third word from the string and sent the updated string to Alex via email.')